# Save all images of video

In [ ]:
import cv2
camera = cv2.VideoCapture("NewObjectDetector.mp4")
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
counter = 0

while camera.isOpened():
    ret, frame = camera.read()
    if ret:

        try:
            image = cv2.resize(frame, (960, 540))
            print(f'Saving Image No. {counter}')
            cv2.imwrite(f'Data/{counter}.bmp', image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        except:
            pass
        counter +=1
    else:
        print('Frame not available')
        break


# Access camera with opencv and tkinter

In [ ]:
# Import required Libraries
from tkinter import *
import tkinter.font as font
from PIL import Image, ImageTk
import cv2

# Create an instance of TKinter Window or frame
win = Tk()
def quit():
        win.destroy()

# Create a Label to capture the Video frames
label =Label(win)
label.grid(row=0, column=0)
cap= cv2.VideoCapture(0)

# Define function to show frame
def show_frames():
   # Get the latest frame and convert into Image
   cv2image= cv2.cvtColor(cap.read()[1],cv2.COLOR_BGR2RGB)
   img = Image.fromarray(cv2image)
   # Convert image to PhotoImage
   imgtk = ImageTk.PhotoImage(image = img)
   label.imgtk = imgtk
   label.configure(image=imgtk)
   # Repeat after an interval to capture continiously
   label.after(20, show_frames)


show_frames()


myFont = font.Font(family='Helvetica', size=15)

button = Button(win, text = 'Click and Quit',  font=myFont ,command=quit)
button.place(relx = 0.4, rely = 0.9, anchor = CENTER)

win.mainloop()





# Save image of certain region and show bound box on realtime camera

In [ ]:
import cv2
import numpy as np
from flask import Flask, render_template, Response
import base64
import cv2
from flask import Flask, jsonify, render_template, request
import numpy as np
import json
app = Flask(__name__)



# Initialize OpenCV face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Capture video stream from webcam
video_capture = cv2.VideoCapture(0)

def detect_faces(frame):
    frame = cv2.flip(frame, 1)

    # Get the dimensions of the frame
    height, width, _ = frame.shape

    # Calculate center coordinates
    center_x = width // 2
    center_y = height // 2

    # Set the width and height of the rectangle
    rect_width = 250
    rect_height = 270

    # Calculate the top-left corner coordinates of the rectangle
    x = center_x - (rect_width // 2)
    y = center_y - (rect_height // 2)
    y = y-40


    # Define the region of interest (ROI)
    roi = frame[y:y+rect_height, x:x+rect_width]
    # roi = frame[x:rect_width, y:rect_height]

    # Save the cropped image
    cv2.imwrite("cropped_image.jpg", roi)
    # Draw the rectangle on the frame
    cv2.rectangle(frame, (x, y), (x + rect_width, y + rect_height), (0, 255, 0), 2)
    return frame


def generate_frames():
    while True:
        # Read frames from the video stream
        ret, frame = video_capture.read()

        # Process frames (detect faces and draw bounding boxes)
        processed_frame = detect_faces(frame)

        # Convert processed frame to JPEG format
        ret, buffer = cv2.imencode('.jpg', processed_frame)
        frame_bytes = buffer.tobytes()

        # Yield the frame bytes as an HTTP response
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n')

@app.route('/')
def index():
    return render_template('index.html')  # Render the HTML page

@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    # Load the face dictionary from JSON file
    with open('face_dict.json', 'r') as json_file:
        face_dict = json.load(json_file)
    print('Embeddings loaded')
    app.run(debug=True)


In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>Real-time Face Detection</title>
</head>
<body>
    <h1>Real-time Face Detection</h1>
    <img src="{{ url_for('video_feed') }}" width="640" height="480">
</body>
</html>
